<a href="https://colab.research.google.com/github/iamBehzad/Advanced-Differential-Evolution/blob/main/MHA_FS_14030908.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install mafese

  Using cached mealpy-2.5.4-py3-none-any.whl.metadata (92 kB)
Using cached mealpy-2.5.4-py3-none-any.whl (377 kB)
  Attempting uninstall: mealpy
    Found existing installation: mealpy 3.0.1
    Uninstalling mealpy-3.0.1:
      Successfully uninstalled mealpy-3.0.1


In [2]:
!pip install mealpy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: mealpy
    Found existing installation: mealpy 2.5.4
    Uninstalling mealpy-2.5.4:
      Successfully uninstalled mealpy-2.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mafese 0.2.0 requires mealpy==2.5.4, but you have mealpy 3.0.1 which is incompatible.


In [1]:
import numpy as np
import mealpy
from mealpy import Optimizer, FloatVar,Multitask, GA

import mafese
from mafese.wrapper.mha import MhaSelector
from mafese import get_dataset

from sklearn.svm import SVC

ImportError: cannot import name 'FloatVar' from 'mealpy' (/usr/local/lib/python3.10/dist-packages/mealpy/__init__.py)

In [4]:
class ADE(Optimizer):
    """
    This is an example how to build new optimizer
    """

    def __init__(self, epoch=10000, pop_size=100, NPopmin=4, pCR=0.75, **kwargs):
        super().__init__(**kwargs)
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [10, 10000])
        self.NPopmin = self.validator.check_int("NPopmin", NPopmin, [4, int(self.pop_size/2)])
        self.pCR = self.validator.check_float("pCR", pCR, (0, 1.0))
        self.lb, self.ub, self.n_dims = None, None, None

        self.sort_flag = True
        # Determine to sort the problem or not in each epoch
        ## if True, the problem always sorted with fitness value increase
        ## if False, the problem is not sorted

    def initialize_variables(self):
      pass
    def initialization(self):
        """
        Override this method if needed. But the first 2 lines of code is required.
        """
        ### Required code
        if self.pop is None:
            self.pop = self.generate_population(self.pop_size)

        ### Your additional code can be implemented here
        self.lb = self.problem.lb
        self.ub = self.problem.ub
        self.n_dims = self.problem.n_dims

        # ADE Parameters
        self.NPopinit = self.pop_size -2
        self.Pop1Size = self.NPopinit
        self.Pop2Size = self.pop_size - self.Pop1Size
        self.beta_min = 0.2  # Lower Bound of Scaling Factor
        self.beta_max = 0.8  # Upper Bound of Scaling Factor

        initial_pop = self.chaotic_initialization(self.pop_size, self.n_dims, self.problem.lb, self.problem.ub)
        self.pop = [self.generate_agent(initial_pop[idx]) for idx in range(0, self.pop_size)]

        # Find initial global best solution
        self.g_best = self.get_sorted_population(self.pop, minmax="min")[0]
        self.pop1 = self.pop[:self.Pop1Size]
        self.pop2 = self.pop[self.Pop1Size:]
        #print("len(self.pop=)",len(self.pop))
        #print("len(self.pop1=)",len(self.pop1))
        #print("len(self.pop2=)",len(self.pop2))

    def evolve(self, epoch):
        self.Pop1Size = round((((self.NPopmin - self.NPopinit) / self.epoch) * (epoch-1)) + self.NPopinit)
        #print (f"self.Pop1Size={self.Pop1Size}", f"self.NPopmin={self.NPopmin}",
        #       f"self.NPopinit={self.NPopinit}", f"self.epoch={self.epoch}",f"epoch={epoch}")
        if self.Pop1Size < len(self.pop1):
            self.Pop2Size = self.pop_size - self.Pop1Size
            self.pop1, self.g_best = self.update_global_best_agent(self.pop1)
            self.pop2.append(self.g_best)  # Add the Best to pop2
            self.pop1=self.pop1[:-1] # Delete the worst from pop1

        # Phase 1: Update pop1
        for i in range(self.Pop1Size):
            x1 = self.pop1[i].solution
            A = np.random.permutation(self.Pop1Size)
            A = np.delete(A, np.where(A == i))
            a, b, c = A[:3]
            # Scale Factor (beta1)
            if np.linalg.norm(self.pop1[b].solution  - self.pop1[c].solution ) > (np.linalg.norm(self.ub - self.lb) / 10):
                beta1 = np.random.uniform(self.beta_min, self.beta_max, self.n_dims)
            else:
                beta1 = np.random.uniform(self.beta_min, self.beta_max * (1 + np.random.rand()), self.n_dims)

            y1 = self.pop1[a].solution + beta1 * (self.pop1[b].solution - self.pop1[c].solution)
            y1 = np.clip(y1, self.lb, self.ub)
            z1 = self.crossover(x1, y1, self.pCR)

            # Evaluate and update best solution
            new_agent = self.generate_agent(z1)
            if self.compare_target(new_agent.target , self.pop1[i].target , minmax="min"):
                self.pop1[i] = new_agent
            #else:
            #    #print([z1, self.n_dims, self.lb, self.ub])
            #    #obl_z1 = self.obl_pos(z1, self.n_dims, self.lb, self.ub)
            #    obl_z1 = self.generate_opposition_solution(new_agent, self.g_best)
            #    obl_agent = self.generate_agent(obl_z1)
            #    if self.compare_target(obl_agent.target , self.pop1[i].target , minmax="min"):
            #        self.pop1[i] = obl_agent

            # Update global best solution
            self.pop1, self.g_best = self.update_global_best_agent(self.pop1)

        # Phase 2: Update pop2
        if self.Pop2Size >= (2 * self.NPopmin):
            self.pop2[:self.NPopmin] = self.pop1[:self.NPopmin]

        for j in range(self.Pop2Size):
            x2 = self.pop2[j].solution
            resOfTop2 = (np.random.rand(self.n_dims) * (x2 - self.pop2[0].solution) +
                         np.random.rand(self.n_dims) * (x2 - self.pop2[1].solution))
            beta2 = np.random.uniform(self.beta_min, self.beta_max, self.n_dims)
            y2 = beta2 * resOfTop2
            y2 = np.clip(y2, self.lb, self.ub)
            z2 = self.crossover(x2, y2, self.pCR)
            new_agent2 = self.generate_agent(z2)
            self.g_best = self.get_better_agent(self.g_best, new_agent2, minmax = "min", reverse= False)
            self.pop2.append(new_agent2)

        # Sort pop2 and retain only Pop2Size solutions
        self.pop2 = self.get_sorted_and_trimmed_population(self.pop2, self.Pop2Size, minmax = "min")


    def logistic_map(self,x, mu=4):
      # Ensure x remains in the valid range
      x = np.clip(x, 0, 1)
      return mu * x * (1 - x)

    def chaotic_initialization(self, pop_size, dim, lb, ub, num_iterations=10):
        pop = []
        lb = np.array(lb)
        ub = np.array(ub)
        x = np.random.rand(dim)  # Initialize within [0, 1]

        for i in range(pop_size):
            for _ in range(num_iterations):  # Apply logistic map multiple times for chaos
                x = self.logistic_map(x)
            # Rescale chaotic values to [lb, ub]
            chaotic_position = lb + x * (ub - lb)
            pop.append(chaotic_position)
        return pop

    def crossover(self,x, y, pCR):
        z = np.zeros_like(x)
        j0 = np.random.randint(0, len(x))
        for jj in range(len(x)):
            if jj == j0 or np.random.rand() <= pCR:
                z[jj] = y[jj]
            else:
                z[jj] = x[jj]
        return z

    def obl_pos(X, dim, lb, ub):
        #lb = np.array(lb)  # Lower bound for each dimension
        #ub = np.array(ub)  # Upper bound for each dimension
        # Calculate the opposite position
        OPos = ub + lb - X  # Element-wise operation
        OPos = np.clip(OPos, lb, ub)
        return OPos

In [5]:
from mafese.wrapper.mha import MhaSelector
from mafese import get_dataset
from sklearn.svm import SVC

data = get_dataset("Arrhythmia")
data.split_train_test(test_size=0.2)
print(data.X_train.shape, data.X_test.shape)            # (361, 279) (91, 279)

optimizer = GA.BaseGA(epoch=100, pop_size=50, pc=0.85, pm=0.1)

# define mafese feature selection method
feat_selector = MhaSelector(problem="classification", estimator="knn",
                            optimizer=optimizer, optimizer_paras=None,
                            transfer_func="vstf_01", obj_name="AS")
# find all relevant features
feat_selector.fit(data.X_train, data.y_train, fit_weights=(0.9, 0.1), verbose=True)

# check selected features - True (or 1) is selected, False (or 0) is not selected
print(feat_selector.selected_feature_masks)
print(feat_selector.selected_feature_solution)

# check the index of selected features
print(feat_selector.selected_feature_indexes)

# call transform() on X to filter it down to selected features
X_train_selected = feat_selector.transform(data.X_train)
X_test_selected = feat_selector.transform(data.X_test)

# Evaluate final dataset with different estimator with multiple performance metrics
results = feat_selector.evaluate(estimator=SVC(), data=data, metrics=["AS", "PS", "RS"])
print(results)
# {'AS_train': 0.77176, 'PS_train': 0.54177, 'RS_train': 0.6205, 'AS_test': 0.72636, 'PS_test': 0.34628, 'RS_test': 0.52747}

Requested CLASSIFICATION dataset: Arrhythmia found and loaded!
(361, 279) (91, 279)


TypeError: Problem.__init__() takes from 2 to 3 positional arguments but 4 were given